In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as spst
import scipy.integrate as spint

from scipy.optimize import newton

import matplotlib.pyplot as plt
import warnings
import os
#import sys
#sys.path.insert(sys.path.index('')+1, 'D:/Github/PyFeng')
import pyfeng as pf

# Basket Option

In [3]:
sigma = 0.4*np.ones(4)
spot = np.ones(4)*100
texp=5

In [4]:
m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
m.simulate(tobs = [texp], n_path=10000)  # tobs: observed t
print(m.n_path)
m.path.shape

10000


(1, 10000, 4)

In [5]:
_ = m.simulate(tobs = [texp], n_path=50000, store=2)
print(m.n_path)
print(m.path.shape)

60000
(1, 60000, 4)


In [6]:
# varying strikes
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)

strikes = np.arange(50, 151, 10)
price = []
for strike in strikes:
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([54.92679384, 48.11095395, 42.16350828, 36.99504987, 32.50356789,
       28.61957621, 25.26155084, 22.34144434, 19.79993616, 17.58970694,
       15.66817728])

In [7]:
# varying forwards
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
strike = 100
price = []
for spot in np.arange(50, 151, 10):
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([ 4.54551749,  7.79239206, 11.9092721 , 16.82164738, 22.42497892,
       28.61957621, 35.33937465, 42.51139716, 50.06198584, 57.93137459,
       66.08222223])

In [8]:
# varying except sigma1=100%
strike = spot = 100
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
price = []
for sigma1 in np.array([5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100])/100:
    sigma = sigma1 * np.ones(4)
    sigma[0] = 1
    #print(sigma)
    m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
    m.simulate(tobs = [texp], n_path=400000)
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([18.94495855, 20.45829493, 22.49840945, 24.87785108, 30.11378007,
       35.57949546, 41.05473026, 46.42608331, 51.61861158, 56.59050047,
       65.9322128 ])

# Simpon's method for integration

Let's integrate exp(-a*t) from 0 to texp

int exp(-a*t) = (1/a)(1 - exp(-a*texp))

In [9]:
a = 10
texp = 2
exact = (1 - np.exp(-a*texp))/a  # analytic solution
exact

0.09999999979388464

In [10]:
def f(t, a):
    return np.exp(-a*t)

n_step = 100
tobs = np.arange(0, n_step+1)/n_step * texp
simp = spint.simps(f(tobs, a), dx=texp/n_step)  # Simpson's method, dx: width of each slice
#simp = spint.simps(f(tobs, a), dx=1) * texp/n_step
print(simp, simp-exact)

# Note: Simpson's method doesn't work very well when the function changes very rapidly, i.e. increase a.

0.10000088446767286 8.846737882123312e-07


# For SABR Model
## Integration of sigma(t)

In [11]:
# You can use BsmNdMc because sigma_t is also a geometric Brownian motion

vov = 0.3
texp = 5
m = pf.BsmNdMc(vov, rn_seed=1234)
tobs = np.arange(0, 101)/100*texp

In [12]:
_ = m.simulate(tobs = tobs, n_path=1000)

In [13]:
print(m.path.shape)
sigma_path = np.squeeze(m.path)
print(sigma_path.shape)

(101, 1000, 1)
(101, 1000)


In [18]:
sigma_final = sigma_path[-1,:]
sigma_final

array([2.42973877, 0.26242663, 1.13376573, 0.56239851, 0.55726737,
       1.14420507, 0.60088606, 1.06114652, 2.52780873, 0.25224541,
       0.69084553, 0.92296776, 1.69857111, 0.37539091, 0.98178759,
       0.64945631, 0.42901648, 1.48625563, 1.58705596, 0.40176791,
       2.02585825, 0.3147447 , 1.59156812, 0.40062888, 0.32577052,
       1.95729237, 0.40875204, 1.55993875, 0.70995952, 0.89811903,
       0.22359618, 2.85169514, 1.12421215, 0.56717778, 0.30860786,
       2.06614357, 0.83272978, 0.76570836, 1.26774928, 0.50296077,
       0.27165773, 2.34717471, 0.20447807, 3.11832047, 0.23856971,
       2.67271206, 0.82583844, 0.77209793, 1.25136435, 0.50954636,
       0.43436512, 1.46795432, 1.31905605, 0.48339732, 0.51654116,
       1.23441887, 1.35309989, 0.47123509, 0.51100029, 1.2478039 ,
       0.80618909, 0.79091637, 1.24100566, 0.51379955, 1.61130496,
       0.39572158, 1.06963191, 0.59611923, 0.74005456, 0.86159614,
       0.67832736, 0.94000063, 0.78027138, 0.81718767, 0.33295

In [20]:
sigma_initial = sigma_path[0,:]
sigma_initial

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [15]:
# compute It
int_var = spint.simps(sigma_path**2, dx=1, axis=0)/100  # axis=0: time dimension
int_var.shape

(1000,)

In [16]:
np.mean(sigma_final)  # close to 1 because sigma(t) is a martingale and sigma(0)=1

1.007675560307406